Illia Bannitsyn illiabann@student.agh.edu.pl

Patrycja Marchwica patmar@student.agh.edu.pl

# Import danych

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


ModuleNotFoundError: No module named 'keras'

# Funkcje

## Deskew



In [ ]:
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

## HOG (Histogram of Oriented Gradients)



In [ ]:
def HOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

## Tworzenie modelu dla sieci


In [ ]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(512, activation='relu', input_shape=(81,)))
  model.add(layers.Dense(10, activation='softmax'))
  model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

## Grid Search - Walidacja krzyżowa

In [ ]:
def SVMGridSearch(train_data, train_labels):
  parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
  svc = svm.SVC()
  clf = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_
  my_svc = clf.best_estimator_
  
  print("Best hiperparameters: ", best_params)
  
  return my_svc

In [ ]:
def RandomForestGridSearch(train_data, train_labels):
  parameters = {
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10],
    'min_samples_split': [2, 5, 10, 15, 100],
    'n_estimators': [20, 30, 50, 70, 100]
  }

  rf = RandomForestClassifier()
  clf = GridSearchCV(estimator = rf, param_grid = parameters, cv = 3, n_jobs = -1, verbose = 1)
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_
  my_rf = clf.best_estimator_
  
  print("Best hiperparameters: ", best_params)

  return my_rf

In [ ]:
def NeuralNetworkGridSrearch(train_data, train_labels):
  parameters = {
      'batch_size': [30, 50, 100],
      'epochs': [5, 10, 15]
      }
  kc = KerasClassifier(build_fn=create_model, verbose=0)
  clf = GridSearchCV(estimator=kc, param_grid=parameters, n_jobs=-1, cv=3)
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_

  print("Best hiperparameters: ", best_params)

  return best_params

## Macierz pomyłek i raporty

In [ ]:
def getConfMatrix(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed):
  cm_deskewed = confusion_matrix(test_labels, pred_labels_deskewed)
  cm_non_deskewed = confusion_matrix(test_labels, pred_labels_non_deskewed)
  cm_raw_deskewed = confusion_matrix(test_labels, pred_labels_raw_deskewed)
  # cm_shuffle = 


  print("Macierz pomyłek - dane wyrównane:")
  print(cm_deskewed)
  print("Macierz pomyłek - dane niewyrównane:")
  print(cm_non_deskewed)
  print("Macierz pomyłek - dane surowe:")
  # print(cm_raw_deskewed)
  print("Macierz pomyłek - dane zmieszane:")
  # print(cm_shuffle)

In [ ]:
def getReports(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed):
  print("Raport - dane wyrównane:")
  print(classification_report(test_labels, pred_labels_deskewed))
  print("Raport - dane niewyrównane:")
  print(classification_report(test_labels, pred_labels_non_deskewed))
  print("Raport - dane surowe:")
  print(classification_report(test_labels, pred_labels_raw_deskewed))
  print("Raport - dane zmieszane:")
  # print()

In [ ]:
def getAccuracy(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed):
  print("Accuracy - dane wyrównane: " + str(accuracy_score(test_labels, pred_labels_deskewed)))
  print("Accuracy - dane niewyrównane: " + str(accuracy_score(test_labels, pred_labels_non_deskewed)))
  print("Accuracy - dane surowe: " + str(accuracy_score(test_labels, pred_labels_raw_deskewed)))
  print("Accuracy - dane zmieszane: ")

# Preprocessing



Surowe dane

In [ ]:
train_raw = train_images.reshape(len(train_images), 28 * 28)
test_raw = test_images.reshape(len(test_images), 28 * 28)

Dane poddane wyrównaniu

In [ ]:
train_deskewed = np.float32([deskew(im) for im in train_raw])
test_deskewed = np.float32([deskew(im) for im in test_raw])

train_deskewed = np.asarray(train_deskewed).reshape(-1,28*28)
test_deskewed = np.asarray(test_deskewed).reshape(-1,28*28)

In [ ]:
hogdata_train = np.float32([HOG(im) for im in train_images]).reshape(-1,81)
hogdata_test = np.float32([HOG(im) for im in test_images]).reshape(-1,81)

hogdata_train_deskewed = np.float32([HOG(deskew(im)) for im in train_images]).reshape(-1,81)
hogdata_test_deskewed = np.float32([HOG(deskew(im)) for im in test_images]).reshape(-1,81)

Dane dla walidacji krzyżowej

In [ ]:
hogdata_train_short = hogdata_train[:600]
hogdata_train_deskewed_short = hogdata_train_deskewed[:600]

# Surowe dane 
rawdata_train_deskewed_short = train_deskewed[:600]

# Zmieszane dane surowe 
# TODO

train_labels_short = train_labels[:600]

# Model SVM

## Walidacja krzyżowa

Dane wyrównane

In [ ]:
model_deskewed = SVMGridSearch(hogdata_train_deskewed_short, train_labels_short)

Dane niewyrównane

In [ ]:
model_non_deskewed = SVMGridSearch(hogdata_train_short, train_labels_short)

Dane surowe

In [ ]:
model_raw_deskewed = SVMGridSearch(rawdata_train_deskewed_short, train_labels_short)

Zmieszane dane surowe

In [ ]:
# TODO

## Trenowanie modelu

Klasyfikacja

In [ ]:
model_deskewed.fit(hogdata_train_deskewed, train_labels)
model_non_deskewed.fit(hogdata_train, train_labels)
model_raw_deskewed.fit(train_deskewed, train_labels)

pred_labels_deskewed = model_deskewed.predict(hogdata_test_deskewed)
pred_labels_non_deskewed = model_non_deskewed.predict(hogdata_test)
pred_labels_raw_deskewed = model_raw_deskewed.predict(test_deskewed)

Accuracy

In [ ]:
getAccuracy(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

Macierze pomyłek

In [ ]:
getConfMatrix(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

Raporty

In [ ]:
getReports(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

## Wnioski

**TODO**


# Model RandomForest

## Walidacja krzyżowa

Dane wyrównane

In [ ]:
model_deskewed = RandomForestGridSearch(hogdata_train_deskewed_short, train_labels_short)

Dane niewyrównane

In [ ]:
model_non_deskewed = RandomForestGridSearch(hogdata_train_short, train_labels_short)

Dane surowe

In [ ]:
model_raw_deskewed = RandomForestGridSearch(rawdata_train_deskewed_short, train_labels_short)

Zmieszane dane surowe

In [ ]:
# TODO

## Trenowanie modelu

Klasyfikacja

In [ ]:
model_deskewed.fit(hogdata_train_deskewed, train_labels)
model_non_deskewed.fit(hogdata_train, train_labels)
model_raw_deskewed.fit(train_deskewed, train_labels)

pred_labels_deskewed = model_deskewed.predict(hogdata_test_deskewed)
pred_labels_non_deskewed = model_non_deskewed.predict(hogdata_test)
pred_labels_raw_deskewed = model_raw_deskewed.predict(test_deskewed)

Accuracy

In [ ]:
getAccuracy(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

Macierz pomyłek

In [ ]:
getConfMatrix(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

Raporty

In [ ]:
getReports(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

# Wnioski

**TODO**

# Model Neural Network

## Stworzenie modelu

In [ ]:
network_deskewed = create_model()
network_non_deskewed = create_model()
network_raw_deskewed = create_model()
# network_shuffle = create_model()

In [ ]:
nn_train_deskewed = np.array(hogdata_train_deskewed).reshape((60000, 81))
nn_train_deskewed = nn_train_deskewed.astype('float32') / 255

nn_test_deskewed = np.array(hogdata_test_deskewed).reshape((10000, 81))
nn_test_deskewed = nn_test_deskewed.astype('float32') / 255

nn_train_non_deskewed = np.array(hogdata_train).reshape((60000, 81))
nn_train_non_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_non_deskewed = np.array(hogdata_test).reshape((10000, 81))
nn_test_non_deskewed = nn_test_non_deskewed.astype('float32') / 255

nn_train_raw_deskewed = np.array(train_deskewed).reshape((60000, 81))
nn_train_raw_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_raw_deskewed = np.array(test_deskewed).reshape((10000, 81))
nn_test_raw_deskewed = nn_test_non_deskewed.astype('float32') / 255

In [ ]:
encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

## Walidacja krzyżowa

Dane wyrównane

In [ ]:
model_deskewed = NeuralNetworkGridSrearch(nn_train_deskewed, encoded_train_labels)

Dane niewyrównane

In [ ]:
model_non_deskewed = NeuralNetworkGridSrearch(nn_train_non_deskewed, encoded_train_labels)

Dane surowe

In [ ]:
model_raw_deskewed = NeuralNetworkGridSrearch(nn_train_raw_deskewed, encoded_train_labels)

Zmieszane dane surowe

In [ ]:
# TODO

## Trenowanie modelu

Dane wyrównane

In [ ]:
network_deskewed.fit(nn_train_deskewed, encoded_train_labels, epochs=model_deskewed.get('epochs'), batch_size=model_deskewed.get('batch_size'))

Dane niewyrównane

In [ ]:
network_non_deskewed.fit(nn_train_non_deskewed, encoded_train_labels, epochs=model_non_deskewed.get('epochs'), batch_size=model_non_deskewed.get('batch_size'))

Dane surowe

In [ ]:
network_deskewed.fit(nn_train_deskewed, encoded_train_labels, epochs=model_raw_deskewed.get('epochs'), batch_size=model_raw_deskewed.get('batch_size'))

Zmieszane dane surowe

In [ ]:
# TODO

In [ ]:
pred_probabilities_deskewed = network_deskewed.predict(nn_test_deskewed)
pred_probabilities_non_deskewed = network_non_deskewed.predict(nn_test_non_deskewed)
pred_probabilities_raw_deskewed = network_non_deskewed.predict(nn_test_raw_deskewed)
# pred_probabilities_shuffle = 

pred_labels_deskewed = np.argmax(pred_probabilities_deskewed,-1)
pred_labels_non_deskewed = np.argmax(pred_probabilities_non_deskewed,-1)
pred_labels_raw_deskewed = np.argmax(pred_probabilities_raw_deskewed,-1)
# pred_labels_shuffle = 

Accuracy

In [ ]:
getAccuracy(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

Macierz pomyłek

In [ ]:
getConfMatrix(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

Raporty

In [ ]:
getReports(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed)

## Wnioski

**TODO**

# Podsumowanie

**TODO**